In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [18]:

# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1, 28, 28, 1])

k = 4
l = 8
m = 12
n = 16
f = 20

def get_weight(shape,regularizer): #shape为列表，regularizer为正则化权重
    w = tf.Variable(tf.truncated_normal(shape,stddev = 0.1))
    tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(regularizer)(w)) #正则化，来缓解过拟合,将所有计算好的正则化了的 w 加在 losses 赋值在上
    return w



#W1 = tf.Variable(tf.truncated_normal([5,5,1,k],stddev = 0.1))
W1 = get_weight([5,5,1,k],0.01)
b1 = tf.Variable(tf.ones([k])/10)

#W2 = tf.Variable(tf.truncated_normal([5,5,k,l],stddev = 0.1))
W2 = get_weight([5,5,k,l],0.01)
b2 = tf.Variable(tf.ones([l])/10)

#W3 = tf.Variable(tf.truncated_normal([5,5,l,m],stddev = 0.1))
W3 = get_weight([5,5,l,m],0.01)
b3 = tf.Variable(tf.ones([m])/10)

W4 = get_weight([3,3,m,n],0.01)
b4 = tf.Variable(tf.ones([n])/10)

W5 = get_weight([1,1,n,f],0.01)
b5= tf.Variable(tf.ones([f])/10)

t = 200  #n

#W4 = tf.Variable(tf.truncated_normal([1*1*m,n],stddev = 0.1))
W6 = get_weight([1*1*f,t],0.01)
b6 = tf.Variable(tf.ones([t])/10)

#W5 = tf.Variable(tf.truncated_normal([n,10],stddev = 0.1))
W7 = get_weight([t,10],0.01)
b7 = tf.Variable(tf.ones([10])/10)

y1 = tf.nn.relu(tf.nn.conv2d(x_image,W1,strides=[1,1,1,1],padding = 'SAME')+b1)
print("y1.shape")
print(y1.shape)
y1 = max_pool(y1)
print(y1.shape)

y2 = tf.nn.relu(tf.nn.conv2d(y1,W2,strides=[1,1,1,1],padding = 'SAME')+b2)
print("y2.shape")
print(y2.shape)
y2 = max_pool(y2)
print(y2.shape)

y3 = tf.nn.relu(tf.nn.conv2d(y2,W3,strides=[1,1,1,1],padding = 'SAME')+b3)
print("y3.shape")
print(y3.shape)
y3 = max_pool(y3)
print(y3.shape)

y4 = tf.nn.relu(tf.nn.conv2d(y3,W4,strides=[1,1,1,1],padding = 'SAME')+b4)
print("y4.shape")
print(y4.shape)
y4 = max_pool(y4)
print(y4.shape)

y5 = tf.nn.relu(tf.nn.conv2d(y4,W5,strides=[1,1,1,1],padding = 'SAME')+b5)
print("y5.shape")
print(y5.shape)
y5 = max_pool(y5)
print(y5.shape)

yy = tf.reshape(y5,shape=[-1,1*1*f])

y6 = tf.nn.relu(tf.matmul(yy,W6)+b6)
print(y6.shape)

y = tf.nn.softmax(tf.matmul(y6, W7) + b7)
print(y.shape)

# 为了计算交叉熵，我们首先需要添加一个新的占位符用于输入正确值
y_ = tf.placeholder("float", [None, 10])

# 交叉熵损失函数
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
cross_function = cross_entropy + tf.add_n(tf.get_collection('losses'))

#训练方法，学习率为1e-4
train_step = tf.train.GradientDescentOptimizer(1e-4).minimize(cross_function)

#初始化变量
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

# 开始训练模型,循环训练1000次
for i in range(20000):
    # 随机抓取训练数据中的100个批处理数据点
    batch_xs, batch_ys = mnist.train.next_batch(100)

    # 喂入数据，开始训练
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})
    if(i%100 == 0):
        print("train step is : "+str(i))
        #计算正确率
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
    

# 检验真实标签与预测标签是否一致
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
# 计算精确度，将true和false转化成相应的浮点数，求和取平均
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
# 计算所学习到的模型在测试数据集上面的正确率
print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
y1.shape
(?, 28, 28, 4)
(?, 14, 14, 4)
y2.shape
(?, 14, 14, 8)
(?, 7, 7, 8)
y3.shape
(?, 7, 7, 12)
(?, 4, 4, 12)
y4.shape
(?, 4, 4, 16)
(?, 2, 2, 16)
y5.shape
(?, 2, 2, 20)
(?, 1, 1, 20)
(?, 200)
(?, 10)
train step is : 0
0.0892
train step is : 100
0.1094
train step is : 200
0.1135
train step is : 300
0.1135
train step is : 400
0.1135
train step is : 500
0.1135
train step is : 600
0.1135
train step is : 700
0.1135
train step is : 800
0.1139
train step is : 900
0.1135
train step is : 1000
0.1141
train step is : 1100
0.144
train step is : 1200
0.1773
train step is : 1300
0.2077
train step is : 1400
0.3065
train step is : 1500
0.3805
train step is : 1600
0.4805
train step is : 1700
0.5032
train step is : 1800
0.65
train step is : 1900
0.6832
train step is : 2000
0.7497
train step is : 2100
0.7618
t